In [1]:
import xarray as xr
import numpy as np
import pandas

In [2]:
def read_csv(csv_ffn, header_line):
    """
    CSV reader used for the radar locations file (comma delimited)
    
    Parameters:
    ===========
        csv_ffn: str
            Full filename to csv file
            
        header_line: int or None
            to use first line of csv as header = 0, use None to use column index
            
    Returns:
    ========
        as_dict: dict
            csv columns are dictionary
    
    """
    df = pandas.read_csv(csv_ffn, header=header_line, skipinitialspace=True)
    as_dict = df.to_dict(orient='list')
    return as_dict

In [3]:
airport_csv_fn = 'airport_table.csv'
era5_ffn = '/g/data/kl02/jss548/era5-data/cape_convrain_sfc_era5.nc'
out_folder = '../preprocessed_data/'
start_date = '20100101'
end_date = '20181231'

In [4]:
#load airport list
ap_dict = read_csv(airport_csv_fn, header_line=1)
ap_name_list = ap_dict['Name']
ap_lat_list = ap_dict['Latitude']
ap_lon_list = ap_dict['Longitude']
ap_rid_list = ap_dict['radar_id']

In [5]:
#ERA5 data

#load netcdf data
DS = xr.open_dataset(era5_ffn)

#find ERA5 cape and crr for each airport
for i, ap_name in enumerate(ap_name_list):
    era5_cape = DS.cape.sel(longitude=ap_lon_list[i], method='nearest').sel(latitude=ap_lat_list[i], method='nearest').sel(time=slice(start_date, end_date))
    era5_crr = DS.crr.sel(longitude=ap_lon_list[i], method='nearest').sel(latitude=ap_lat_list[i], method='nearest').sel(time=slice(start_date, end_date))
    era5_time = DS.time.sel(time=slice(start_date, end_date))
    #save to file
    save_path = out_folder + ap_name + '_era5.npz'
    np.savez(save_path, era5_cape=era5_cape[:], era5_crr=era5_crr[:], era5_time=era5_time[:])
    print('finished', ap_name)

finished MEL
finished SYD
finished WSYD
finished BNE
